In [2]:
from collections import defaultdict
from pathlib import Path

input_data = Path("../input_data/08").read_text()

In [ ]:
# {
#     "a": ["a", "b", ...,  "g"],
#     "c": ["a"],
#     "d": ["c", "f"],
#     "f": ["c", "f"],
# }


a -> g
b -> b
c -> a
d -> f
e -> e
f -> c
g -> d


df (1)  -> (cf)(cf)
c -> a (aan de hand van de 7)
dcf (7) -> (cf)(a)(cf)
gfbd (4)

cadbg
gaefc
agdcf
abcdeg (ontbreekt f)
abcdfg (ontbreekt e)
abcdef (ontbreekt g)

(efg) -> cde
| acgbd dafcg fd df

een van de twee cijfers van 1 zit in alle cijfers van zes lang (6, 0, 9) (de f in 1 met default signal)
hetzelfde met de b in alle cijfers van zes lang (6, 0, 9) die uit de 4 komt

In [3]:
default_display_signals = {
    1: {          "c",           "f"     }, # f komt voor in alle #6

    7: {"a",      "c",           "f"     }, # 7 - 1 = a

    4: {     "b", "c", "d",      "f"     },  # 4 - 1 -> bd  b komt voor in alle #6
                                             # g komt in alles van 6 voor
                                             # e komt niet in alles van 6 voor

    2: {"a",      "c", "d", "e",      "g"},  # Niet nodig
    3: {"a",      "c", "d",      "f", "g"},  # Niet nodig
    5: {"a", "b",      "d",      "f", "g"},  # Niet nodig

    6: {"a", "b",      "d", "e", "f", "g"},
    0: {"a", "b", "c",      "e", "f", "g"},
    9: {"a", "b", "c", "d",      "f", "g"},

    8: {"a", "b", "c", "d", "e", "f", "g"},
}


available_signals = {"a", "b", "c", "d", "e", "f", "g"}

In [44]:
all_numbers = []
for line in input_data.split("\n"):
    signal_conversion_options = {sig: available_signals.copy() for sig in available_signals}
    signals, outputs = line.split("|")
    signals_by_len = defaultdict(list)
    for signal in signals.split():
        signals_by_len[len(signal)].append(set(signal))

    cf = signals_by_len[2][0]
    cdf = signals_by_len[3][0]
    bcdf = signals_by_len[4][0]

    a = cdf - cf
    bd = bcdf - cf

    # Get the intersection of all signals of length 6
    abcdefg = signals_by_len[7][0]  # The signal with len 7 has all signals
    six_intersection = abcdefg.intersection(*signals_by_len[6])
    abfg = six_intersection  # The six intersection contains abfg

    # The b signal is in the 6 intersection
    b = abfg & bd
    d = bd - b

    # The f signal is in the 6 intersection
    f = six_intersection & cf
    c = cf - f

    # Just e and g left
    eg = abcdefg - a - b - c - d - f
    # g is in the 6 intersection
    g = abfg & eg
    e = eg - g


    remap_to_default = {
        a.pop(): "a",
        b.pop(): "b",
        c.pop(): "c",
        d.pop(): "d",
        e.pop(): "e",
        f.pop(): "f",
        g.pop(): "g"
    }

    numbers = []
    for output_signal in outputs.split(" "):

        remapped_signal = set(remap_to_default[x] for x in output_signal)
        for number, dds in default_display_signals.items():
            if dds == remapped_signal:
                numbers.append(number)

    all_numbers.append(numbers)

In [46]:
all_numbers[:10]

[[5, 3, 1, 1],
 [1, 7, 3, 0],
 [8, 6, 8, 7],
 [4, 8, 4, 2],
 [5, 1, 5, 8],
 [4, 1, 0, 6],
 [8, 1, 8, 0],
 [7, 0, 1, 1],
 [7, 8, 2, 3],
 [1, 0, 1, 7]]

In [47]:
from collections import Counter
import itertools

flat_number_list = list(itertools.chain(*all_numbers))
c = Counter(flat_number_list)
c

Counter({5: 44,
         3: 44,
         1: 132,
         7: 140,
         0: 57,
         8: 137,
         6: 55,
         4: 112,
         2: 42,
         9: 37})

In [49]:
# part one result
c[1] + c[4] + c[7] + c[8]

521

In [52]:
# part two result
total_sum = 0
for number_list in all_numbers:
    concat_number = int("".join(str(number) for number in number_list))
    total_sum += concat_number

total_sum

1016804